In [9]:
import os
import random
import shutil
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2, ResNet50, VGG16, EfficientNetB0
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, GlobalAveragePooling2D, Input, Dropout
from sklearn.metrics import confusion_matrix, classification_report

In [10]:
# Paths
original_base = r'C:\Users\gajje\OneDrive\Desktop\Major\Lung_diagnosis_files\Dataset\Dataset\Dataset\round-1\Level1\10%Base'
original_inbox = r'C:\Users\gajje\OneDrive\Desktop\Major\Lung_diagnosis_files\Dataset\Dataset\Dataset\round-1\Level2\Inbox'
original_outbox = r'C:\Users\gajje\OneDrive\Desktop\Major\Lung_diagnosis_files\Dataset\Dataset\Dataset\round-1\Level3\Outbox'
train_dir = 'temp_dataset/train'
test1_dir = 'temp_dataset/test1'
test2_dir = 'temp_dataset/test2'
test3_dir = 'temp_dataset/test3'

# Clear and make folders
for folder in [train_dir, test1_dir, test2_dir, test3_dir]:
    shutil.rmtree(folder, ignore_errors=True)
    os.makedirs(folder, exist_ok=True)

# Copying function
def split_and_copy(src_folder, dst_folder, split_ratio=1.0, split_type='random'):
    labels = os.listdir(src_folder)
    for label in labels:
        src_label_path = os.path.join(src_folder, label)
        dst_label_path = os.path.join(dst_folder, label)
        os.makedirs(dst_label_path, exist_ok=True)
        files = [f for f in os.listdir(src_label_path)
                 if os.path.isfile(os.path.join(src_label_path, f)) and f.lower().endswith(('.jpg', '.jpeg', '.png'))]
        files.sort()
        if split_type == 'random':
            files = random.sample(files, int(len(files) * split_ratio))
        elif split_type == 'first_half':
            files = files[:int(len(files) * split_ratio)]
        elif split_type == 'second_half':
            files = files[int(len(files) * split_ratio):]
        for f in files:
            shutil.copy(os.path.join(src_label_path, f), os.path.join(dst_label_path, f))

# Split and copy
split_and_copy(original_base, train_dir, 0.9)
split_and_copy(original_base, test1_dir, 0.1)
split_and_copy(original_inbox, test2_dir)
split_and_copy(original_outbox, train_dir, 0.5, 'first_half')
split_and_copy(original_outbox, test3_dir, 0.5, 'second_half')

# Data Generators
img_size = (128, 128)
batch_size = 16
datagen = ImageDataGenerator(rescale=1./255)

train_data = datagen.flow_from_directory(train_dir, target_size=img_size, batch_size=batch_size, class_mode='categorical')
test1_data = datagen.flow_from_directory(test1_dir, target_size=img_size, batch_size=batch_size, class_mode='categorical', shuffle=False)
test2_data = datagen.flow_from_directory(test2_dir, target_size=img_size, batch_size=batch_size, class_mode='categorical', shuffle=False)
test3_data = datagen.flow_from_directory(test3_dir, target_size=img_size, batch_size=batch_size, class_mode='categorical', shuffle=False)

num_classes = train_data.num_classes

Found 441 images belonging to 3 classes.
Found 12 images belonging to 3 classes.
Found 534 images belonging to 3 classes.
Found 324 images belonging to 3 classes.


In [14]:
# Helper to build base + classifier
def build_model(base_model):
    base_model.trainable = False
    model = Sequential([
        base_model,
        GlobalAveragePooling2D(),
        Dense(128, activation='relu'),
        Dropout(0.3),
        Dense(num_classes, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# Models
model_custom = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(128,128,3)),
    MaxPooling2D(2,2),
    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    Conv2D(128, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    Flatten(),
    Dense(64, activation='relu'),
    Dense(num_classes, activation='softmax')
])
model_custom.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model_mobilenet = build_model(MobileNetV2(input_shape=(128,128,3), include_top=False, weights='imagenet'))
model_resnet = build_model(ResNet50(input_shape=(128,128,3), include_top=False, weights='imagenet'))
model_vgg = build_model(VGG16(input_shape=(128,128,3), include_top=False, weights='imagenet'))
model_efficient = build_model(EfficientNetB0(input_shape=(128,128,3), include_top=False, weights='imagenet'))

# AlexNet (custom build)
model_alexnet = Sequential([
    Conv2D(96, (11,11), strides=(4,4), activation='relu', input_shape=(128,128,3)),
    MaxPooling2D(3, strides=2),
    Conv2D(256, (5,5), padding='same', activation='relu'),
    MaxPooling2D(3, strides=2),
    Conv2D(384, (3,3), padding='same', activation='relu'),
    Conv2D(384, (3,3), padding='same', activation='relu'),
    Conv2D(256, (3,3), padding='same', activation='relu'),
    MaxPooling2D(3, strides=2),
    Flatten(),
    Dense(4096, activation='relu'),
    Dropout(0.5),
    Dense(4096, activation='relu'),
    Dropout(0.5),
    Dense(num_classes, activation='softmax')
])
model_alexnet.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


C:\Users\gajje\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [16]:
# Train all models (printing model name before training starts)
EPOCHS = 10

print("\n🔄 Training Custom CNN...")
model_custom.fit(train_data, epochs=EPOCHS, validation_data=test1_data)

print("\n🔄 Training MobileNetV2...")
model_mobilenet.fit(train_data, epochs=EPOCHS, validation_data=test1_data)

print("\n🔄 Training ResNet50...")
model_resnet.fit(train_data, epochs=EPOCHS, validation_data=test1_data)

print("\n🔄 Training VGG16...")
model_vgg.fit(train_data, epochs=EPOCHS, validation_data=test1_data)

print("\n🔄 Training EfficientNetB0...")
model_efficient.fit(train_data, epochs=EPOCHS, validation_data=test1_data)

print("\n🔄 Training AlexNet...")
model_alexnet.fit(train_data, epochs=EPOCHS, validation_data=test1_data)



🔄 Training Custom CNN...
Epoch 1/10
28/28 ━━━━━━━━━━━━━━━━━━━━ 4s 147ms/step - accuracy: 0.9979 - loss: 0.0349 - val_accuracy: 1.0000 - val_loss: 0.0520
Epoch 2/10
28/28 ━━━━━━━━━━━━━━━━━━━━ 3s 121ms/step - accuracy: 1.0000 - loss: 0.0167 - val_accuracy: 1.0000 - val_loss: 0.0287
Epoch 3/10
28/28 ━━━━━━━━━━━━━━━━━━━━ 4s 128ms/step - accuracy: 1.0000 - loss: 0.0102 - val_accuracy: 1.0000 - val_loss: 0.0079
Epoch 4/10
28/28 ━━━━━━━━━━━━━━━━━━━━ 4s 130ms/step - accuracy: 1.0000 - loss: 0.0039 - val_accuracy: 1.0000 - val_loss: 0.0049
Epoch 5/10
28/28 ━━━━━━━━━━━━━━━━━━━━ 4s 129ms/step - accuracy: 1.0000 - loss: 0.0019 - val_accuracy: 1.0000 - val_loss: 0.0030
Epoch 6/10
28/28 ━━━━━━━━━━━━━━━━━━━━ 4s 126ms/step - accuracy: 1.0000 - loss: 0.0010 - val_accuracy: 1.0000 - val_loss: 0.0022
Epoch 7/10
28/28 ━━━━━━━━━━━━━━━━━━━━ 4s 127ms/step - accuracy: 1.0000 - loss: 8.9526e-04 - val_accuracy: 1.0000 - val_loss: 0.0016
Epoch 8/10
28/28 ━━━━━━━━━━━━━━━━━━━━ 4s 131ms/step - accuracy: 1.0000 - l

In [ ]:
# Evaluate all models
model_accuracies = {}

def evaluate_model(name, model, test_data):
    loss, acc = model.evaluate(test_data, verbose=0)
    model_accuracies[name] = acc
    print(f"✅ {name} Accuracy: {acc:.4f}")

evaluate_model("Custom CNN", model_custom, test3_data)
evaluate_model("MobileNetV2", model_mobilenet, test3_data)
evaluate_model("ResNet50", model_resnet, test3_data)
evaluate_model("VGG16", model_vgg, test3_data)
evaluate_model("EfficientNetB0", model_efficient, test3_data)
evaluate_model("AlexNet", model_alexnet, test3_data)

print("\n📊 All Model Accuracies (Outbox Test Set):")
for name, acc in model_accuracies.items():
    print(f"{name}: {acc:.4f}")

In [11]:
# 🔁 Choose model: 'Custom_CNN', 'MobileNetV2', 'ResNet50', 'VGG16', 'EfficientNetB0', 'AlexNet'
model_name = 'MobileNetV2'

# Model builder
def build_model(name):
    input_shape = (128, 128, 3)
    
    if name == 'Custom_CNN':
        model = Sequential([
            Conv2D(32, (3,3), activation='relu', input_shape=input_shape),
            MaxPooling2D(2,2),
            Conv2D(64, (3,3), activation='relu'),
            MaxPooling2D(2,2),
            Conv2D(128, (3,3), activation='relu'),
            MaxPooling2D(2,2),
            Flatten(),
            Dense(64, activation='relu'),
            Dense(num_classes, activation='softmax')
        ])
    
    elif name == 'MobileNetV2':
        base = MobileNetV2(weights='imagenet', include_top=False, input_shape=input_shape)
        base.trainable = False
        x = GlobalAveragePooling2D()(base.output)
        x = Dense(64, activation='relu')(x)
        x = Dropout(0.5)(x)
        output = Dense(num_classes, activation='softmax')(x)
        model = Model(inputs=base.input, outputs=output)
    
    elif name == 'ResNet50':
        base = ResNet50(weights='imagenet', include_top=False, input_shape=input_shape)
        base.trainable = False
        x = GlobalAveragePooling2D()(base.output)
        x = Dense(64, activation='relu')(x)
        x = Dropout(0.5)(x)
        output = Dense(num_classes, activation='softmax')(x)
        model = Model(inputs=base.input, outputs=output)
    
    elif name == 'VGG16':
        base = VGG16(weights='imagenet', include_top=False, input_shape=input_shape)
        base.trainable = False
        x = GlobalAveragePooling2D()(base.output)
        x = Dense(64, activation='relu')(x)
        x = Dropout(0.5)(x)
        output = Dense(num_classes, activation='softmax')(x)
        model = Model(inputs=base.input, outputs=output)
    
    elif name == 'EfficientNetB0':
        base = EfficientNetB0(weights='imagenet', include_top=False, input_shape=input_shape)
        base.trainable = False
        x = GlobalAveragePooling2D()(base.output)
        x = Dense(64, activation='relu')(x)
        x = Dropout(0.5)(x)
        output = Dense(num_classes, activation='softmax')(x)
        model = Model(inputs=base.input, outputs=output)

    elif name == 'AlexNet':
        model = Sequential([
            Conv2D(96, (11,11), strides=(4,4), activation='relu', input_shape=input_shape),
            MaxPooling2D((3,3), strides=(2,2)),
            Conv2D(256, (5,5), padding='same', activation='relu'),
            MaxPooling2D((3,3), strides=(2,2)),
            Conv2D(384, (3,3), padding='same', activation='relu'),
            Conv2D(384, (3,3), padding='same', activation='relu'),
            Conv2D(256, (3,3), padding='same', activation='relu'),
            MaxPooling2D((3,3), strides=(2,2)),
            Flatten(),
            Dense(4096, activation='relu'),
            Dropout(0.5),
            Dense(4096, activation='relu'),
            Dropout(0.5),
            Dense(num_classes, activation='softmax')
        ])
    
    else:
        raise ValueError("Model not recognized!")
    
    return model

In [12]:
# Build, compile and train
model = build_model(model_name)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

history = model.fit(train_data, epochs=10, validation_data=test1_data)

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 128, 128,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Conv1 (Conv2D)      │ (None, 64, 64,    │        864 │ input_layer[0][0] │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bn_Conv1            │ (None, 64, 64,    │        128 │ Conv1[0][0]       │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Conv1_relu (ReLU)   │ (None, 64, 64,    │          0 │ bn_Conv1[0][0]    │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 64, 64,    │        288 │ Conv1_relu[0][0]  │
│ (DepthwiseConv2D)   │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 64, 64,    │        128 │ expanded_conv_de… │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 64, 64,    │          0 │ expanded_conv_de… │
│ (ReLU)              │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_proj… │ (None, 64, 64,    │        512 │ expanded_conv_de… │
│ (Conv2D)            │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_proj… │ (None, 64, 64,    │         64 │ expanded_conv_pr… │
│ (BatchNormalizatio… │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand      │ (None, 64, 64,    │      1,536 │ expanded_conv_pr… │
│ (Conv2D)            │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand_BN   │ (None, 64, 64,    │        384 │ block_1_expand[0… │
│ (BatchNormalizatio… │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand_relu │ (None, 64, 64,    │          0 │ block_1_expand_B… │
│ (ReLU)              │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_pad         │ (None, 65, 65,    │          0 │ block_1_expand_r… │
│ (ZeroPadding2D)     │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise   │ (None, 32, 32,    │        864 │ block_1_pad[0][0] │
│ (DepthwiseConv2D)   │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise_… │ (None, 32, 32,    │        384 │ block_1_depthwis… │
│ (BatchNormalizatio… │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise_… │ (None, 32, 32,    │          0 │ block_1_depthwis… │
│ (ReLU)              │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_project     │ (None, 32, 32,    │      2,304 │ block_1_depthwis

 Total params: 2,340,163 (8.93 MB)

 Trainable params: 82,179 (321.01 KB)

 Non-trainable params: 2,257,984 (8.61 MB)

C:\Users\gajje\AppData\Roaming\Python\Python312\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
28/28 ━━━━━━━━━━━━━━━━━━━━ 17s 171ms/step - accuracy: 0.3299 - loss: 1.4979 - val_accuracy: 0.4167 - val_loss: 1.0659
Epoch 2/10
28/28 ━━━━━━━━━━━━━━━━━━━━ 3s 108ms/step - accuracy: 0.4170 - loss: 1.0710 - val_accuracy: 0.5000 - val_loss: 1.1152
Epoch 3/10
28/28 ━━━━━━━━━━━━━━━━━━━━ 4s 132ms/step - accuracy: 0.4596 - loss: 1.0567 - val_accuracy: 0.5000 - val_loss: 1.0459
Epoch 4/10
28/28 ━━━━━━━━━━━━━━━━━━━━ 5s 182ms/step - accuracy: 0.4375 - loss: 1.0486 - val_accuracy: 0.4167 - val_loss: 1.0895
Epoch 5/10
28/28 ━━━━━━━━━━━━━━━━━━━━ 8s 296ms/step - accuracy: 0.5158 - loss: 1.0226 - val_accuracy: 0.4167 - val_loss: 1.0849
Epoch 6/10
28/28 ━━━━━━━━━━━━━━━━━━━━ 8s 273ms/step - accuracy: 0.4625 - loss: 1.0262 - val_accuracy: 0.5833 - val_loss: 1.0952
Epoch 7/10
28/28 ━━━━━━━━━━━━━━━━━━━━ 9s 314ms/step - accuracy: 0.5109 - loss: 0.9978 - val_accuracy: 0.4167 - val_loss: 1.0519
Epoch 8/10
28/28 ━━━━━━━━━━━━━━━━━━━━ 10s 343ms/step - accuracy: 0.5412 - loss: 0.9749 - val_accuracy: 

In [17]:

# Evaluate all models
model_accuracies = {}

def evaluate_model(name, model, test_data):
    loss, acc = model.evaluate(test_data, verbose=0)
    model_accuracies[name] = acc
    print(f"✅ {name} Accuracy: {acc:.4f}")

evaluate_model("Custom CNN", model_custom, test3_data)
evaluate_model("MobileNetV2", model_mobilenet, test3_data)
evaluate_model("ResNet50", model_resnet, test3_data)
evaluate_model("VGG16", model_vgg, test3_data)
evaluate_model("EfficientNetB0", model_efficient, test3_data)
evaluate_model("AlexNet", model_alexnet, test3_data)

print("\n All Model Accuracies (Outbox Test Set):")
for name, acc in model_accuracies.items():
    print(f"{name}: {acc:.4f}")

✅ Custom CNN Accuracy: 0.6327
✅ MobileNetV2 Accuracy: 0.5463
✅ ResNet50 Accuracy: 0.3920
✅ VGG16 Accuracy: 0.4321
✅ EfficientNetB0 Accuracy: 0.3889
✅ AlexNet Accuracy: 0.3889

📊 All Model Accuracies (Outbox Test Set):
Custom CNN: 0.6327
MobileNetV2: 0.5463
ResNet50: 0.3920
VGG16: 0.4321
EfficientNetB0: 0.3889
AlexNet: 0.3889
